<a href="https://colab.research.google.com/github/sherkhalil/COVID19/blob/master/BERT_Classifier_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
dataset = '/content/drive/My Drive/Jan9-2012-tweets-clean (version 1).csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd 
data = pd.read_csv(dataset) 
data.head()

,Id,Tweet,Label
0,145353048817012000,Thinks that @melbahughes had a great 50th birt...,surprise
1,144279638024257000,"Como una expresiÃ³n tan simple, una sola oraci...",sadness
2,140499585285111000,the moment when you get another follower and y...,joy
3,145207578270507000,Be the greatest dancer of your life! practice ...,joy
4,139502146390470000,eww.. my moms starting to make her annual rum ...,disgust


In [ ]:
data.Label.value_counts()

joy         8240
surprise    3849
sadness     3830
fear        2816
anger       1555
disgust      761
Name: Label, dtype: int64

In [ ]:
#data.drop(data.loc[data.Label == 'joy'].index, inplace=True)
#data.drop(data.loc[data.Label == 'surprise'].index, inplace=True)

data.drop(data.loc[data.Label == 'sadness'].index, inplace=True)
data.drop(data.loc[data.Label == 'fear'].index, inplace=True)
data.drop(data.loc[data.Label == 'anger'].index, inplace=True)
data.drop(data.loc[data.Label == 'disgust'].index, inplace=True)
data.Label.value_counts()

joy         8240
surprise    3849
Name: Label, dtype: int64

In [ ]:
data['Label_Numeric'] = data['Label'].map({'joy':0, 'surprise':1})

In [ ]:
data.Label_Numeric.value_counts()

0    8240
1    3849
Name: Label_Numeric, dtype: int64

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40kB 1.5MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.5-cp36-none-any.whl size=30317 sha256=12e62bf4e0a675c7910fe71d5a4218003d46f4a12f21daf42c0d7e39a89f0716
  Stored in directory: /root/.cache/pip/wheels/2e/70/a2/be357037dd2cbdcaeb0add1fdf083be6a600ca65ee1f68751c
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7304 sha256=a2e88b4e07fa72699064797ba46953f0b5172f1ae7c4a8a3434212ef4c139ea9
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19475 sha256=74624570c243ac504664b7e695d38da57e7e2c3f0a194ac2c2abd9ee4af4585e
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 2.8MB/s 


In [ ]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [ ]:
data.shape

(12089, 4)

In [ ]:
data['Tweet']

0        Thinks that @melbahughes had a great 50th birt...
2        the moment when you get another follower and y...
3        Be the greatest dancer of your life! practice ...
5        If ur heart hurts all the time for tht person ...
6        I feel awful, and it's way too freaking early....
                               ...                        
21044                            Accident on mouse creek. 
21046    @TheBodyShopUK Knowing my dissertation will be...
21047    hospital tomorrow morning, strapped with wires...
21048    Work is soooo slow, ready to have a great satu...
21049    You realize that by choosing joy every single ...
Name: Tweet, Length: 12089, dtype: object

In [ ]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [ ]:
import re
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [ ]:
reviews = []
sentences = list(data['Tweet'])
for sen in sentences:
    reviews.append(preprocess_text(sen))

In [ ]:
print(data.columns.values)

['Id' 'Tweet' 'Label' 'Label_Numeric']


In [ ]:
data.Label_Numeric.unique()

array([1, 0])

In [ ]:
import numpy as np
y = data['Label_Numeric']

y = np.array(list(map(lambda x: 1 if x==1 else 0, y)))

In [ ]:
y

array([1, 0, 0, ..., 0, 0, 0])

In [ ]:
reviews[10]

'I have package at the post office Can think what could be in it don remember internet shopping while drinking '

In [ ]:
y[10]

1

In [ ]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [ ]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [ ]:
tokenized_reviews = [tokenize_reviews(review) for review in reviews]

In [ ]:
tokenized_reviews[0]

[6732, 2008, 11463, 24206, 8953, 2229, 2018, 2307, 16215, 5798, 2283]

In [ ]:
reviews_with_len = [[review, y[i], len(review)]  for i, review in enumerate(tokenized_reviews)]

In [ ]:
import random
random.shuffle(reviews_with_len)

In [ ]:
reviews_with_len.sort(key=lambda x: x[2])

In [ ]:
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]

In [ ]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

In [ ]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [ ]:
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 3), dtype=int32, numpy=
 array([[ 2650,  3113,  4826],
        [ 2155,  3185,  2051],
        [ 3169,  4234,  2775],
        [ 3178,  2482,  4536],
        [12252,  4234,  7534],
        [ 4234,  2557,  2276],
        [ 2397,  2305,  6023],
        [ 2146,  5353,  3805],
        [ 2147,  6229,  2485],
        [ 4234,  3392,  6023],
        [25935,  3419, 15854],
        [ 3313,  2558,  2671],
        [ 4234,  7700,  2651],
        [ 2147,  6229, 11501],
        [ 2437, 13921, 23839],
        [ 2204,  2851,  2088],
        [ 2188,  4086,  2188],
        [10135, 10080,  3892],
        [ 3178,  5670,  4826],
        [ 6467,  4611,  4826],
        [11208,  3231,  4826],
        [ 2188,  4086,  2188],
        [ 2147,  2147,  2147],
        [27233,  8320,  2651],
        [ 7083,  3116,  3892],
        [ 5983,  9485, 11942],
        [ 2482,  4536,  2188],
        [ 2293,  5223,  3276],
        [ 2829,  4524,  2569],
        [ 4172,  9458,  2051],
        [ 3635,  2731,

In [ ]:
import math
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [ ]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="BERT_Classifier_B"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [ ]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [ ]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [ ]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [ ]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
341/341 [==============================] - 33s 96ms/step - loss: 0.4979 - accuracy: 0.7562
Epoch 2/5
341/341 [==============================] - 29s 85ms/step - loss: 0.2677 - accuracy: 0.8912
Epoch 3/5
341/341 [==============================] - 29s 85ms/step - loss: 0.0823 - accuracy: 0.9729
Epoch 4/5
341/341 [==============================] - 29s 85ms/step - loss: 0.0317 - accuracy: 0.9892
Epoch 5/5
341/341 [==============================] - 29s 85ms/step - loss: 0.0268 - accuracy: 0.9909


In [ ]:
results = text_model.evaluate(test_data)
print(results)

37/37 [==============================] - 0s 8ms/step - loss: 0.9560 - accuracy: 0.7812
[0.9559502601623535, 0.78125]


In [ ]:
text_model.summary()


Model: "BERT_Classifier_B"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  6104400   
_________________________________________________________________
conv1d_3 (Conv1D)            multiple                  40100     
_________________________________________________________________
conv1d_4 (Conv1D)            multiple                  60100     
_________________________________________________________________
conv1d_5 (Conv1D)            multiple                  80100     
_________________________________________________________________
global_max_pooling1d_1 (Glob multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple                  77056     
_________________________________________________________________
dropout_1 (Dropout)          multiple            